# Building Models with PyTorch 使用 PyTorch 构建模型

观看[youtube视频](https://www.youtube.com/watch?v=OSqIP-mOWOI)

## `torch.nn.Module`和`torch.nn.Parameter`

除了`Parameter`之外，我们在本视频中讨论的类都是`torch.nn.Module`的子类。这是 PyTorch 基类，旨在封装特定于 PyTorch 模型及其组件的行为。

`torch.nn.Module` 的一个重要行为是注册参数。如果某个 `Module` 子类有可学习的权重，这些权重将以 `torch.nn.Parameter` 的实例形式表达。`Parameter` 类是 `torch.Tensor` 的一个子类，具有特殊的行为，当它们被赋值为 `Module` 的属性时，它们会被添加到该模块参数的列表中。这些参数可以通过 `Module` 类的 `parameters()` 方法访问。

作为一个简单的示例，这是一个非常简单的模型，具有两个线性层和一个激活函数。我们将创建它的一个实例并要求它报告其参数：

In [1]:
import torch

class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(100, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

tinymodel = TinyModel()

print('The model:')
print(tinymodel)

print('\n\nJust one layer:')
print(tinymodel.linear2)

print('\n\nModel params:')
for param in tinymodel.parameters():
    print(param)

print('\n\nLayer params:')
for param in tinymodel.linear2.parameters():
    print(param)

The model:
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


Just one layer:
Linear(in_features=200, out_features=10, bias=True)


Model params:
Parameter containing:
tensor([[ 0.0531,  0.0988, -0.0950,  ...,  0.0311, -0.0429, -0.0383],
        [ 0.0002, -0.0691, -0.0142,  ...,  0.0029,  0.0947,  0.0861],
        [-0.0268, -0.0805, -0.0084,  ...,  0.0696, -0.0580,  0.0044],
        ...,
        [-0.0798, -0.0020,  0.0541,  ...,  0.0503,  0.0480, -0.0565],
        [ 0.0680, -0.0802, -0.0850,  ..., -0.0637, -0.0496,  0.0311],
        [-0.0917,  0.0846, -0.0071,  ..., -0.0973, -0.0055,  0.0023]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0823, -0.0691,  0.0086,  0.0860,  0.0702,  0.0149,  0.0980, -0.0237,
         0.0579,  0.0569,  0.0969, -0.0621, -0.0915, -0.0289, -0.0831, -0.0524,
         0.0316,  0.0612, -0.0591,  0.09

这显示了 PyTorch 模型的基本结构：有一个`__init__()`方法定义模型的层和其他组件，以及一个完成计算的`forward()`方法。请注意，我们可以打印模型或其任何子模块，以了解其结构。

## 常见层类型

### 线性层

神经网络中最基本的层类型是线性层或全连接层。这是一种层，在该层中每个输入都会影响该层的每个输出，影响的程度由该层的权重决定。如果一个模型有 `m` 个输入和 `n` 个输出，那么权重将是一个 `m x n` 的矩阵。例如：

In [2]:
lin = torch.nn.Linear(3, 2)
x = torch.rand(1, 3)
print('Input:')
print(x)

print(lin.weight)
print('\n\nWeight and Bias parameters:')
for param in lin.parameters():
    print(param)

y = lin(x)
print('\n\nOutput:')
print(y)

Input:
tensor([[0.1554, 0.4020, 0.0627]])
Parameter containing:
tensor([[ 0.1950, -0.1878, -0.0288],
        [ 0.1975, -0.4694, -0.4273]], requires_grad=True)


Weight and Bias parameters:
Parameter containing:
tensor([[ 0.1950, -0.1878, -0.0288],
        [ 0.1975, -0.4694, -0.4273]], requires_grad=True)
Parameter containing:
tensor([ 0.1044, -0.2500], requires_grad=True)


Output:
tensor([[ 0.0574, -0.4348]], grad_fn=<AddmmBackward0>)


如果你将输入 `x` 与线性层的权重矩阵相乘，并加上偏置，你会发现得到的就是输出向量 `y`。

另一个重要的特性需要注意：当我们使用 `lin.weight` 检查我们层的权重时，它报告了自己是一个 `Parameter`（`Parameter` 是 `Tensor` 的子类），并让我们知道它正在使用 autograd 跟踪梯度。这是 `Parameter` 的默认行为，与 `Tensor` 不同。

线性层在深度学习模型中被广泛使用。你会经常在分类器模型中看到它们，这些模型通常在最后包含一个或多个线性层，最后一层将有 `n` 个输出，其中 `n` 是分类器要处理的类别数。

### 卷积层

**卷积层** 是为处理具有高度空间相关性的数据而设计的。它们非常常见于计算机视觉领域，在那里它们检测特征的紧密分组，并将这些特征组成更高级别的特征。它们也出现在其他上下文中——例如，在自然语言处理（NLP）应用中，一个词的上下文（即，序列中附近的其他词）可以影响句子的意义。

我们在之前的视频中看到过 LeNet5 中卷积层的应用：

In [3]:
import torch.functional as F


class LeNet(torch.nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel (black & white), 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = torch.nn.Conv2d(1, 6, 5)
        self.conv2 = torch.nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = torch.nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

让我们来解析这个模型的卷积层发生了什么，从 `conv1` 开始：

- LeNet5 是为了处理 1x32x32 的黑白图像设计的。**卷积层构造函数的第一个参数是输入通道的数量**。这里，它是 1。如果我们要构建一个处理三色通道的模型，它将是 3。

- 卷积层就像一个扫描图像的窗口，寻找它识别的模式。这些模式称为特征，卷积层的一个参数是我们希望它学习的特征数量。**构造函数的第二个参数是输出特征的数量**。在这里，我们要求我们的层学习 6 个特征。

- 刚才，我把卷积层比作一个窗口——但是窗口有多大呢？**第三个参数是窗口或核的大小**。这里，“5” 意味着我们选择了一个 5x5 的核。（如果你想要一个高度不同于宽度的核，可以为此参数指定一个元组，例如 `(3, 5)` 来得到一个 3x5 的卷积核）。

卷积层的输出是一个激活图（activation map）——这是输入张量中特征存在的空间表示。`conv1` 将为我们提供一个形状为 6x28x28 的输出张量；6 是特征数量，28 是激活图的高度和宽度。（当用 5 像素的窗口扫描 32 像素的行时，只有 28 个有效位置。）

然后我们将卷积的输出通过一个 ReLU 激活函数（稍后会详细介绍激活函数），然后通过一个最大池化层（max pooling layer）。最大池化层将激活图中彼此接近的特征组合在一起。它通过减少张量来实现这一点，将输出中的每个 2x2 细胞组合并为一个单元，并将该单元分配给进入其中的 4 个细胞的最大值。这样我们得到了一个较低分辨率的激活图，维度为 6x14x14。

我们的下一个卷积层 `conv2` 期望有 6 个输入通道（对应于第一层所学习的 6 个特征），具有 16 个输出通道，以及一个 3x3 的卷积核。它输出一个 16x12x12 的激活图，这个激活图再次被一个最大池化层减少到 16x6x6。在将此输出传递给线性层之前，它会被重塑为一个 16 * 6 * 6 = 576 元素的向量，以供下一层使用。

有用于处理 1D、2D 和 3D 张量的卷积层。卷积层构造函数还可以接受许多其他可选参数，包括步幅长度（例如，仅在输入中每隔一个或每隔两个位置扫描），填充（这样你可以将扫描扩展到输入的边缘），等等。请参阅[文档]([documentation](https://pytorch.org/docs/stable/nn.html#convolution-layers))以获取更多信息。

### 循环层

**循环神经网络**（Recurrent neural networks，或 RNNs）用于处理序列数据——从科学仪器的时间序列测量数据到自然语言句子再到 DNA。RNN 通过维护一个**隐藏状态**（hidden state）来实现这一点，该状态起到一种记忆作用，用于记录到目前为止在序列中看到的内容。

RNN 层的内部结构——或其变体 LSTM（长短期记忆，long short-term memory）和 GRU（门控循环单元，gated recurrent unit）——相对复杂，超出了本视频的范围，但我们会展示一个基于 LSTM 的词性标注器的实际使用情况（词性标注器是一种分类器，用于告诉你一个词是名词、动词等）。

In [4]:
class LSTMTagger(torch.nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

构造函数有四个参数：

- `vocab_size` 是输入词汇表中的词的数量。每个词是一个 one-hot 向量（或单位向量），位于一个 `vocab_size` 维的空间中。
- `tagset_size` 是输出集中标签的数量。
- `embedding_dim` 是词汇表的嵌入空间大小。嵌入将词汇表映射到一个低维空间，在这个空间中，具有相似含义的词彼此接近。
- `hidden_dim` 是 LSTM 内存的大小。

输入将是一个句子，句子中的单词表示为 one-hot 向量的索引。嵌入层将这些索引映射到一个 `embedding_dim` 维的空间。LSTM 接收这个嵌入序列并迭代处理它，生成一个长度为 `hidden_dim` 的输出向量。最后的线性层充当分类器；将 `log_softmax()` 应用于最终层的输出，会将输出转换为一组归一化的估计概率，表示一个特定的单词与一个特定标签的对应关系。

如果你想看到这个网络的实际运行，请查看 PyTorch 网站上的 [序列模型和 LSTM 网络](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html) 教程。

### Transformers

Transformers 是多用途网络，已成为 NLP 领域的先进模型，如 BERT。关于 transformer 架构的讨论超出了本视频的范围，但 PyTorch 提供了一个 `Transformer` 类，该类允许你定义 transformer 模型的整体参数——注意力头的数量、编码器和解码器层的数量、dropout 和激活函数等。（你甚至可以使用这个类并设置正确的参数来构建 BERT 模型！）`torch.nn.Transformer` 类还包含封装了各个组件（`TransformerEncoder`、`TransformerDecoder`）和子组件（`TransformerEncoderLayer`、`TransformerDecoderLayer`）的类。详情请查看关于 transformer 类的 [文档](https://pytorch.org/docs/stable/nn.html#transformer-layers) 和 PyTorch 网站上的相关 [教程](https://pytorch.org/tutorials/)。

## 其他层和功能

### 数据处理层

还有一些其他层类型在模型中执行重要的功能，但它们本身并不参与学习过程。

**最大池化**（及其对应的最小池化）通过组合单元来减少张量的大小，并将输入单元的最大值分配给输出单元（我们已经看到过这个过程）。例如：

In [5]:
my_tensor = torch.rand(1, 6, 6)
print(my_tensor)

maxpool_layer = torch.nn.MaxPool2d(3)
print(maxpool_layer(my_tensor))

tensor([[[0.0401, 0.2001, 0.4475, 0.6086, 0.6608, 0.2783],
         [0.5288, 0.7100, 0.7741, 0.8455, 0.1099, 0.6405],
         [0.5994, 0.1071, 0.0436, 0.8469, 0.4861, 0.7324],
         [0.4876, 0.8885, 0.6688, 0.5981, 0.5444, 0.0929],
         [0.0671, 0.5476, 0.8226, 0.7248, 0.3336, 0.9359],
         [0.8063, 0.3790, 0.0205, 0.0228, 0.1583, 0.0032]]])
tensor([[[0.7741, 0.8469],
         [0.8885, 0.9359]]])


如果你仔细观察上面的值，你会发现最大池化输出中的每个值都是6x6输入中每个象限的最大值。

**归一化层**在将一个层的输出传递给另一个层之前，会重新居中并归一化这些输出。对中间张量进行居中和缩放有许多有益的效果，比如让你能够在不出现梯度爆炸/消失的情况下使用更高的学习率。

In [6]:
my_tensor = torch.rand(1, 4, 4) * 20 + 5
print(my_tensor)

print(my_tensor.mean())

norm_layer = torch.nn.BatchNorm1d(4)
normed_tensor = norm_layer(my_tensor)
print(normed_tensor)

print(normed_tensor.mean())

tensor([[[ 7.6485,  6.6786, 13.0347, 15.1961],
         [13.1008, 23.4915, 11.1822,  7.0374],
         [21.5202,  7.9166, 22.3819, 20.6254],
         [21.1592, 20.3152,  6.0250, 21.0655]]])
tensor(14.8987)
tensor([[[-0.8365, -1.1078,  0.6699,  1.2744],
         [-0.0993,  1.6149, -0.4159, -1.0997],
         [ 0.5760, -1.7225,  0.7216,  0.4248],
         [ 0.6252,  0.4939, -1.7298,  0.6107]]],
       grad_fn=<NativeBatchNormBackward0>)
tensor(-1.4901e-07, grad_fn=<MeanBackward0>)


运行上面的代码时，我们向输入张量添加了一个大的缩放因子和偏移量；你应该会看到输入张量的 `mean()` 值大约在 15 附近。经过归一化层处理后，你会发现这些值变得更小，并集中在 0 附近——实际上，均值应该非常小（大约 > 1e-8）。

这样做的好处在于，许多激活函数（稍后讨论）在接近 0 的位置具有最强的梯度，但有时当输入值远离 0 时会遇到梯度消失或爆炸的问题。将数据保持在最陡峭梯度的区域附近，通常意味着学习速度更快，学习效果更好，并且可以使用更高的可行学习率。

**Dropout 层** 是一种推动模型生成稀疏表示的工具 —— 也就是说，它鼓励模型在更少的数据下进行推理。

**Dropout 层** 通过在训练过程中随机将输入张量的部分单元设为 0 来工作 —— dropout 层在推理时总是关闭的。这迫使模型在这种屏蔽或减少的数据集下进行学习。例如：

In [7]:
my_tensor = torch.rand(1, 4, 4)

dropout = torch.nn.Dropout(p=0.4)
print(dropout(my_tensor))
print(dropout(my_tensor))

tensor([[[0.0000, 0.0000, 0.0000, 1.2594],
         [1.5891, 0.6768, 0.4402, 0.6474],
         [1.0724, 0.0000, 0.0000, 0.7248],
         [1.5158, 0.0000, 0.1564, 0.7947]]])
tensor([[[0.0000, 0.0000, 1.0218, 1.2594],
         [0.0000, 0.6768, 0.4402, 0.6474],
         [0.0000, 0.0000, 1.5882, 0.0000],
         [1.5158, 0.9061, 0.1564, 0.7947]]])


如上所示，你可以看到 dropout 对样本张量的影响。你可以使用可选的 `p` 参数来设置单个权重被丢弃的概率；如果不设置，它默认为 0.5。

### 激活函数 (Activation Functions)

激活函数使深度学习成为可能。神经网络实际上是一个程序——拥有许多参数——它模拟了一个**数学函数**。如果我们只是在层权重上多次进行张量运算，我们只能模拟**线性函数**；进一步来说，如果没有激活函数，那么整个网络的作用将会被简化为一次矩阵乘法。在线性层之间插入**非线性**的激活函数，才使得深度学习模型能够模拟任何函数，而不仅仅是线性函数。

`torch.nn.Module` 中包含了所有主要的激活函数对象，包括 ReLU 及其变种，Tanh、Hardtanh、Sigmoid 等。此外，还包括其他功能，例如 Softmax，这在模型的输出阶段非常有用。

### 损失函数 (Loss Functions)

损失函数告诉我们模型的预测与正确答案之间的差距。PyTorch 包含多种损失函数，包括常见的 MSE（均方误差 = L2 范数）、交叉熵损失（Cross Entropy Loss）和负似然损失（Negative Likelihood Loss，适用于分类器）等。